# Verse mappings



In [1]:
#r "nuget:SIL.Scripture,10.1.0"
//#r "nuget:SIL.Scripture,11.0.1"
#r "nuget:Thot,3.3.5"
#r "../src/ClearBible.Engine/bin/Debug/net6.0/ClearBible.Engine.dll"
#r "../../machine/src/SIL.Machine/bin/Debug/netstandard2.0/Clear.SIL.Machine.dll"
#r "../../machine/src/SIL.Machine.Translation.Thot/bin/Debug/netstandard2.0/SIL.Machine.Translation.Thot.dll"

using SIL.Machine.Corpora;
using SIL.Machine.Tokenization;
using SIL.Machine.Translation;
using SIL.Machine.Translation.Thot;
using static SIL.Machine.Corpora.TokenProcessors;
using SIL.Machine.Utils;
using SIL.Scripture;

using ClearBible.Engine.Translation;
using ClearBible.Engine.Corpora;
using ClearBible.Engine.Persistence;
using ClearBible.Engine.Tokenization;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages SIL.Scripture, 10.1.0 Thot, 3.3.5

## Generating verse mappings for entire bible, both English versification

In [2]:
var verseMappings = EngineParallelTextCorpus.VerseMappingsForAllVerses(ScrVers.English, ScrVers.English);

var verseMappingsWithMoreThanOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() > 1) || (vm.TargetVerses.Count() > 1));
    
var verseMappingsWithOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() == 1) && (vm.TargetVerses.Count() == 1));

var verseMappingsWithSourceTargetEqual = verseMappings
    .Where(vm => vm.SourceVerses.Count() == 1 && vm.TargetVerses.Count() == 1 && vm.SourceVerses[0].Equals(vm.TargetVerses[0]));
    
Console.WriteLine($"Count of versemappings: {verseMappings.Count()}");
Console.WriteLine($"Count of versemappings with more than one verse: { verseMappingsWithMoreThanOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with one verse: { verseMappingsWithOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with source verse equalling target verse: { verseMappingsWithSourceTargetEqual.Count()}");

Count of versemappings: 38393
Count of versemappings with more than one verse: 0
Count of versemappings with one verse: 38393
Count of versemappings with source verse equalling target verse: 38393


## Generating verse mappings for entire bible, English to Original versification

In [3]:
var verseMappings = EngineParallelTextCorpus.VerseMappingsForAllVerses(ScrVers.English, ScrVers.Original);

var verseMappingsWithMoreThanOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() > 1) || (vm.TargetVerses.Count() > 1));
    
var verseMappingsWithOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() == 1) && (vm.TargetVerses.Count() == 1));

var verseMappingsWithSourceTargetEqual = verseMappings
    .Where(vm => vm.SourceVerses.Count() == 1 && vm.TargetVerses.Count() == 1 && vm.SourceVerses[0].Equals(vm.TargetVerses[0]));
    
Console.WriteLine($"Count of versemappings: {verseMappings.Count()}");
Console.WriteLine($"Count of versemappings with more than one verse: { verseMappingsWithMoreThanOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with one verse: { verseMappingsWithOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with source verse equalling target verse: { verseMappingsWithSourceTargetEqual.Count()}");

Count of versemappings: 38510
Count of versemappings with more than one verse: 3
Count of versemappings with one verse: 38507
Count of versemappings with source verse equalling target verse: 35740


## Custom versification

In [9]:
using System.IO;

Versification.Table.Implementation.RemoveAllUnknownVersifications();
string customVersificationAddition = "&MAT 1:2 = MAT 1:1\nMAT 1:3 = MAT 1:2\nMAT 1:1 = MAT 1:3\n";
ScrVers customEnglishVersification;
using (var reader = new StringReader(customVersificationAddition))
{
    customEnglishVersification = Versification.Table.Implementation.Load(reader, "vers.txt", ScrVers.English, "custom");
}

var verseMappings = EngineParallelTextCorpus.VerseMappingsForAllVerses(ScrVers.English, customEnglishVersification);

var verseMappingsWithMoreThanOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() > 1) || (vm.TargetVerses.Count() > 1));
    
var verseMappingsWithOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() == 1) && (vm.TargetVerses.Count() == 1));

var verseMappingsWithSourceTargetEqual = verseMappings
    .Where(vm => vm.SourceVerses.Count() == 1 &&
        vm.TargetVerses.Count() == 1 && 
        vm.SourceVerses[0].Equals(vm.TargetVerses[0]));
    
Console.WriteLine($"Count of versemappings: {verseMappings.Count()}");
Console.WriteLine($"Count of versemappings with more than one verse: { verseMappingsWithMoreThanOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with one verse: { verseMappingsWithOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with source verse equalling target verse: { verseMappingsWithSourceTargetEqual.Count()}");

Console.WriteLine();
Console.WriteLine("Verse mappings with more than one verse:");
foreach (var verseMapping in verseMappingsWithMoreThanOneVerse)
{
    Console.WriteLine($"{verseMapping}");
    Console.WriteLine($"  SourceVerses:");
    foreach(var verse in verseMapping.SourceVerses)
    {
        Console.WriteLine($"    {verse}");
    }
    Console.WriteLine($"  TargetVerses:");
    foreach(var verse in verseMapping.TargetVerses)
    {
        Console.WriteLine($"    {verse}");
    }
}

Console.WriteLine();
Console.WriteLine("Verse mappings where source does not equal target:");
foreach (var verseMapping in verseMappings
    .Where(vm => vm.SourceVerses.Count() == 1 &&
        vm.TargetVerses.Count() == 1 && 
        !vm.SourceVerses[0].Equals(vm.TargetVerses[0])))
{
    Console.WriteLine($"{verseMapping}");
    Console.WriteLine($"  SourceVerses:");
    foreach(var verse in verseMapping.SourceVerses)
    {
        Console.WriteLine($"    {verse}");
    }
    Console.WriteLine($"  TargetVerses:");
    foreach(var verse in verseMapping.TargetVerses)
    {
        Console.WriteLine($"    {verse}");
    }
}


Count of versemappings: 38379
Count of versemappings with more than one verse: 11
Count of versemappings with one verse: 38368
Count of versemappings with source verse equalling target verse: 38365

Verse mappings with more than one verse:
ESG 4:21
  SourceVerses:
    ESG 4:21
  TargetVerses:
    ESG 4:20
    ESG 4:21
ESG 4:23
  SourceVerses:
    ESG 4:23
  TargetVerses:
    ESG 4:22
    ESG 4:23
ESG 4:31
  SourceVerses:
    ESG 4:31
  TargetVerses:
    ESG 4:29
    ESG 4:30
    ESG 4:31
ESG 4:35
  SourceVerses:
    ESG 4:35
  TargetVerses:
    ESG 4:34
    ESG 4:35
ESG 4:37
  SourceVerses:
    ESG 4:37
  TargetVerses:
    ESG 4:36
    ESG 4:37
ESG 5:4
  SourceVerses:
    ESG 5:4
  TargetVerses:
    ESG 5:2
    ESG 5:3
    ESG 5:4
ESG 5:10
  SourceVerses:
    ESG 5:10
  TargetVerses:
    ESG 5:9
    ESG 5:10
ESG 5:12
  SourceVerses:
    ESG 5:12
  TargetVerses:
    ESG 5:11
    ESG 5:12
ESG 5:14
  SourceVerses:
    ESG 5:14
  TargetVerses:
    ESG 5:13
    ESG 5:14
ESG 5:16
  SourceVer